In [1]:
from github import Github
import pandas as pd

In [2]:
%run settings.ipynb

token = %env token

github = Github(token)

env: token=edfaf32358498c685c2484fbaf84d29b99a15ec1


## Análise de testes usados nos projetos do dataset.

1. Análise de quantos projetos usam arquivos de controle.
2. Armazenar arquivos de controle.
3. Verificar se existe algum pacote de teste no arquivo de controle.
4. Verificar qual pacote está sendo usado.


In [14]:
tools = pd.read_csv('dataset/de_para_tools.csv', sep=';')

tools['language'].unique()


array(['JavaScript', 'Python', 'Java', 'Go', 'Ruby', 'PHP', 'TypeScript',
       'C#'], dtype=object)

In [15]:
manager_file = {}

for t in tools['language'].unique():
    manager_file[t] = {"yes": 0, "total": 0}

manager_file

{'C#': {'total': 0, 'yes': 0},
 'Go': {'total': 0, 'yes': 0},
 'Java': {'total': 0, 'yes': 0},
 'JavaScript': {'total': 0, 'yes': 0},
 'PHP': {'total': 0, 'yes': 0},
 'Python': {'total': 0, 'yes': 0},
 'Ruby': {'total': 0, 'yes': 0},
 'TypeScript': {'total': 0, 'yes': 0}}

In [16]:
manager_package = {}

for t in tools['language'].unique():
    manager_package[t] = {}
    for p in tools[tools['language'] == t]['test_package']:
        manager_package[t][p] = 0

manager_package

{'C#': {'CoreClr': 0, 'NUnit': 0, 'NUnit3TestAdapter': 0},
 'Go': {'Testify': 0,
  'assert': 0,
  'biff': 0,
  'check': 0,
  'convey': 0,
  'e2e': 0,
  'ginkgo': 0,
  'go-check': 0,
  'go-testing-interface': 0,
  'gocheck': 0,
  'gomega': 0,
  'gucumber': 0,
  'require': 0,
  'spew': 0,
  'suite': 0,
  'testing': 0,
  'testutil': 0},
 'Java': {'EasyMock': 0,
  'hamcrest-all': 0,
  'junit': 0,
  'mockito-all': 0,
  'mockito-core': 0,
  'scalatest': 0,
  'spring-test': 0,
  'testNG': 0},
 'JavaScript': {'Appium': 0,
  'Cypress': 0,
  'Electron': 0,
  'Gemini': 0,
  'Happo': 0,
  'Istanbul': 0,
  'Nightwatch': 0,
  'Percy': 0,
  'PhantomJS': 0,
  'Protractor': 0,
  'Puppeteer': 0,
  'Sinon.JS': 0,
  'TestCafe': 0,
  'Unexpected': 0,
  'ava': 0,
  'chai': 0,
  'cucumber': 0,
  'cypress': 0,
  'enzyme': 0,
  'enzyme-adapter-react-16': 0,
  'enzyme-adapter-utils': 0,
  'jest': 0,
  'jest-cli': 0,
  'jsdom': 0,
  'json-server': 0,
  'karma': 0,
  'mocha': 0,
  'nyc': 0,
  'protractor': 0,
  '

### TO-DO: Atualização Dataset.

Filtro para restringir dataset para apenas a lista de linguagens definidas.

In [17]:
repo_full = pd.read_csv('dataset/repositories.csv', sep=';')
#repo_full.drop("Unnamed: 0", axis=1, inplace=True) 
repo_full.head()

,full_name,git_url,id,language
0,b4winckler/macvim,git://github.com/b4winckler/macvim.git,692798,C
1,php/php-src,git://github.com/php/php-src.git,1903522,C
2,antirez/redis,git://github.com/antirez/redis.git,156018,C
3,libgit2/libgit2,git://github.com/libgit2/libgit2.git,901662,C
4,torvalds/linux,git://github.com/torvalds/linux.git,2325298,C


### Quantidade total de projetos de uma mesma linguagem.

In [18]:
for index, row in repo_full.iterrows():
    if row['language'] in manager_file:
        manager_file[row['language']]["total"]+=1

manager_file

{'C#': {'total': 13, 'yes': 0},
 'Go': {'total': 33, 'yes': 0},
 'Java': {'total': 38, 'yes': 0},
 'JavaScript': {'total': 81, 'yes': 0},
 'PHP': {'total': 21, 'yes': 0},
 'Python': {'total': 52, 'yes': 0},
 'Ruby': {'total': 23, 'yes': 0},
 'TypeScript': {'total': 15, 'yes': 0}}

#### Arquivo de controle de pacotes de teste.

In [19]:
tools[tools['language'] == 'Ruby']

,language,test_package,file_packages_installed
68,Ruby,rspec,Gemfile
69,Ruby,Test::Unit,Gemfile
70,Ruby,Cucumber,Gemfile
71,Ruby,Shoulda,Gemfile


### Quantidade total de projetos que possui o arquivo de gerencia de pacotes.

In [22]:
import requests
import csv

resp = []
for index, row in repo_full.iterrows():
    packages = tools[tools['language'] == row['language']]
    arq = packages['file_packages_installed'].unique()
    if(row['language'] == 'Go'):
        if(len(arq) > 0):
            #Pega o repositório pelo nome.
            repo = github.get_repo(row['full_name'])
            #print(row['full_name'])
            try:
                #Pegar um arquivo específico.
                content = repo.get_contents(str(arq[0]))
                print(row['full_name'])
                #Registrar quantidade de projetos possui arquivo de controle.
                manager_file[row['language']]["yes"]+=1

                #Obter conteúdo do arquivo fazendo requisicao direta a API.
                #print('https://raw.githubusercontent.com/' + row['full_name'] + '/master/' +  content.path)
                response = requests.get('https://raw.githubusercontent.com/' + row['full_name'] + '/master/' +  content.path)
                response.encoding = 'utf-8'
                texto = response.text

                #Verificar se possui um dos pacotes da lista de controle.
                is_test = 0
                for p in packages['test_package']:
                    if(texto.count(p)>0):
                        is_test = 1
                        manager_package[row['language']][p]+=1
                        r =  pd.DataFrame([{
                            'full_name': row['full_name'], 
                            'language': row['language'], 
                            'package': p
                        }])
                        resp.append(r)
                if(not is_test):
                    r =  pd.DataFrame([{
                        'full_name': row['full_name'], 
                        'language': row['language'], 
                        'package': None
                    }])
                resp.append(r)

            except:
                #print(row['full_name'], row['language'], arq)
                pass


#manager_file
#manager_package
resp

kubernetes/kubernetes
influxdata/influxdb
hashicorp/terraform
etcd-io/etcd
hashicorp/packer
mattermost/mattermost-server
ipfs/go-ipfs
hashicorp/vault
hashicorp/consul
pingcap/tidb
go-gitea/gitea
git-lfs/git-lfs
istio/istio
prometheus/prometheus
minio/minio
syncthing/syncthing


[               full_name language  package
 0  kubernetes/kubernetes       Go  testing,
                full_name language package
 0  kubernetes/kubernetes       Go  gomega,
                full_name language package
 0  kubernetes/kubernetes       Go  ginkgo,
                full_name language package
 0  kubernetes/kubernetes       Go  assert,
                full_name language package
 0  kubernetes/kubernetes       Go  convey,
                full_name language package
 0  kubernetes/kubernetes       Go   check,
                full_name language package
 0  kubernetes/kubernetes       Go    spew,
                full_name language package
 0  kubernetes/kubernetes       Go    spew,
              full_name language  package
 0  influxdata/influxdb       Go  testing,
              full_name language package
 0  influxdata/influxdb       Go  gomega,
              full_name language package
 0  influxdata/influxdb       Go  ginkgo,
              full_name language package
 0  influx

In [10]:
package_df = pd.concat(resp, ignore_index=True)

In [11]:
package_df.shape

(608, 3)

In [23]:
package_df = package_df.drop_duplicates(subset=['full_name', 'language', 'package'])

In [24]:
package_df.shape

(472, 3)

In [25]:
package_df.to_csv('dataset/miner_package.csv', sep=';', encoding = 'utf-8')

In [98]:
manager_package

{'C#': {'CoreClr': 0, 'NUnit': 0, 'NUnit3TestAdapter': 0},
 'Go': {'Testify': 0,
  'assert': 48,
  'biff': 0,
  'check': 60,
  'convey': 48,
  'e2e': 24,
  'ginkgo': 60,
  'go-check': 16,
  'go-testing-interface': 36,
  'gocheck': 4,
  'gomega': 64,
  'gucumber': 0,
  'require': 0,
  'spew': 65,
  'suite': 4,
  'testing': 55,
  'testutil': 4},
 'Java': {'EasyMock': 0,
  'hamcrest-all': 12,
  'junit': 70,
  'mockito-all': 4,
  'mockito-core': 28,
  'scalatest': 8,
  'spring-test': 8,
  'testNG': 0},
 'JavaScript': {'Appium': 0,
  'Cypress': 0,
  'Electron': 0,
  'Gemini': 4,
  'Happo': 0,
  'Istanbul': 0,
  'Nightwatch': 0,
  'Percy': 0,
  'PhantomJS': 0,
  'Protractor': 0,
  'Puppeteer': 0,
  'Sinon.JS': 0,
  'TestCafe': 0,
  'Unexpected': 0,
  'ava': 82,
  'chai': 105,
  'cucumber': 0,
  'cypress': 4,
  'enzyme': 48,
  'enzyme-adapter-react-16': 36,
  'enzyme-adapter-utils': 4,
  'jest': 92,
  'jest-cli': 44,
  'jsdom': 59,
  'json-server': 0,
  'karma': 81,
  'mocha': 126,
  'nyc': 4

In [13]:
manager_file

{'C#': {'total': 13, 'yes': 7},
 'Go': {'total': 33, 'yes': 16},
 'Java': {'total': 38, 'yes': 21},
 'JavaScript': {'total': 81, 'yes': 73},
 'PHP': {'total': 21, 'yes': 19},
 'Python': {'total': 52, 'yes': 16},
 'Ruby': {'total': 23, 'yes': 17},
 'TypeScript': {'total': 15, 'yes': 15}}

In [99]:
import requests
response = requests.get('https://raw.githubusercontent.com/octokit/octokit.rb/master/Gemfile')
response.encoding = 'utf-8'
texto = response.text
print(texto)
print(texto.count("gemspec"))
#file_response = open("dataset/manager_file/teste.txt","w+")
#file_response.write(texto, "w+")
#file_response.close()


KeyboardInterrupt: 

IOError: [Errno 2] File miner_test.csv does not exist: 'miner_test.csv'